<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import sys
!{sys.executable} -m pip install pyspark==3.1.2 -q
!{sys.executable} -m pip install spark-nlp==4.2.0 -q

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
df = pd.read_csv('drive/MyDrive/696/text_difficulty_features.csv',sep='\t')
df.head(5)

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
0,There is manuscript evidence that Austen conti...,There be manuscript evidence that Austen conti...,8.533221,5.809310,3.57,12.12,2.495517,1.33,4.57,38.0,1
1,"In a remarkable comparative analysis , Mandaea...","In a remarkable comparative analysis , Mandaea...",12.320171,7.402308,2.89,11.94,2.334286,1.46,4.93,21.0,1
2,"Before Persephone was released to Hermes , who...","Before Persephone be release to Hermes , who h...",5.931500,5.231351,2.78,11.17,2.556486,1.43,4.86,40.0,1
3,Cogeneration plants are commonly found in dist...,Cogeneration plant be commonly find in distric...,7.015012,6.742000,3.56,11.53,3.369655,1.52,4.93,32.0,1
4,Geneva is the second-most-populous city in Swi...,Geneva be the second-most-populous city in Swi...,7.550745,5.455000,3.69,12.62,2.399333,1.43,4.79,22.0,1


In [133]:
df.dtypes

original_text        object
lemmatized_text      object
d_chall_score       float64
aoa_mean            float64
aoa_min             float64
aoa_max             float64
conc_rating_mean    float64
conc_rating_min     float64
conc_rating_max     float64
num_lemmas          float64
label                 int64
dtype: object

In [134]:
# Extract only the columns we'll use for the classifier
df = df[df.columns[1:]]

In [135]:
from sklearn.model_selection import train_test_split

In [136]:
X = df[df.columns[:-1]]
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=.1,
                                                    stratify=y,
                                                    random_state=99)

In [137]:
X_train.shape

(355652, 9)

In [162]:
metrics_df = {"model":[],
              "accuracy": [],
              "precision": [],
              "recall": [],
              "aucprc":[],
              "aucroc": []}

In [139]:
from sklearn. metrics import precision_score, recall_score, precision_recall_curve, roc_auc_score

In [140]:
from sklearn.metrics._plot.precision_recall_curve import plot_precision_recall_curve


In [159]:
def ComputeMetrics(model,model_name, X_test_vec, y_test):
    # extract the model's hyperparameters and make that part of the identifier for the model 

    y_pred = model.predict(X_test_vec)
    acc = model.score(X_test_vec, y_test)
    recall = recall_score(y_test,y_pred)
    precision = precision_score(y_test, y_pred)
    y_proba = model.predict_proba(X_test_vec)[:,1]
    prc = precision_recall_curve(y_test, y_proba)
    roc = roc_auc = roc_auc_score(y_test, y_proba)
    
    metrics_df['model'].append(model_name)
    metrics_df['accuracy'].append(acc)
    metrics_df['recall'].append(recall)
    metrics_df['precision'].append(precision)
    metrics_df['aucprc'].append(prc)
    metrics_df['aucroc'].append(roc)








In [142]:
# NaiveBayes Classifier Baseline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [143]:
tf_vec = TfidfVectorizer(min_df = 20,
                         stop_words='english',
                         ngram_range=(1, 3))

In [144]:
X_train_vec = tf_vec.fit_transform(X_train['lemmatized_text'])

In [145]:
from sklearn.model_selection import GridSearchCV


In [146]:
param_grid = {'alpha': [0.001, 0.01, 0.1, 1,5,10,50, 100]}
grid_mnb_rmse = GridSearchCV(MultinomialNB(), param_grid=param_grid, scoring='neg_root_mean_squared_error')
grid_mnb_rmse.fit(X_train_vec,y_train)

GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100]},
             scoring='neg_root_mean_squared_error')

In [147]:
grid_mnb_rmse.cv_results_

{'mean_fit_time': array([0.12147403, 0.11792874, 0.11915994, 0.12117   , 0.11926694,
        0.11890988, 0.11797061, 0.11770577]),
 'std_fit_time': array([0.00467478, 0.00257535, 0.00395118, 0.00426927, 0.00392524,
        0.00318547, 0.00180999, 0.0064425 ]),
 'mean_score_time': array([0.01006641, 0.00967059, 0.01021485, 0.01150126, 0.00993452,
        0.0114264 , 0.00973139, 0.00953913]),
 'std_score_time': array([0.00018257, 0.00013461, 0.00059266, 0.00270246, 0.00019408,
        0.00302299, 0.00017854, 0.0001601 ]),
 'param_alpha': masked_array(data=[0.001, 0.01, 0.1, 1, 5, 10, 50, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.001},
  {'alpha': 0.01},
  {'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 5},
  {'alpha': 10},
  {'alpha': 50},
  {'alpha': 100}],
 'split0_test_score': array([-0.61250873, -0.6124743 , -0.61216435, -0.61048558, -0.60837482,
        -0.60935613, -0.63

In [148]:
# Looks like alpha of 5 is the best parameter based on rmse
grid_mnb_rmse.best_params_

{'alpha': 5}

In [163]:
mnb = MultinomialNB(alpha=5).fit(X_train_vec, y_train)
model_name = 'Tf-idf NB alpha 5'
X_test_vec = tf_vec.transform(X_test['lemmatized_text'])
ComputeMetrics(mnb,model_name,X_test_vec, y_test)



In [164]:
metrics_df

{'model': ['Tf-idf NB alpha 5'],
 'accuracy': [0.6302350886960042],
 'precision': [0.6189553564220553],
 'recall': [0.7477811372281283],
 'aucprc': [(array([0.53178081, 0.53176867, 0.53178246, ..., 1.        , 1.        ,
          1.        ]),
   array([1.        , 0.99995123, 0.99995123, ..., 0.00214571, 0.00204818,
          0.        ]),
   array([0.13515109, 0.13575168, 0.13589074, ..., 0.99989217, 0.99989711,
          0.99992042]))],
 'aucroc': [0.6960772870858367]}

In [150]:
y_pred = mnb.predict(X_test_vec)

((39517,), (39517, 29772))

In [120]:
X_train_vec

<355652x29772 sparse matrix of type '<class 'numpy.float64'>'
	with 4030659 stored elements in Compressed Sparse Row format>